In [0]:
from tqdm import tqdm
from model.nn import *
from pyspark.sql import functions as F
from components.config import *
from components.helper import *

In [0]:
property_name = "Borgata"

In [0]:
path = "data_science_mart.tierimminent_cleaned."
table_name = f"{property_name}"+"_trip_cleaned_return"
cleaned_df = spark.table(f"{path}{table_name}")

unsucess_id = cleaned_df.groupby('train_guest_id').agg(F.count('change_assigned').alias('count')).where('count = 0').sample(withReplacement=False, fraction=0.5, seed=269)
train_final = cleaned_df.join(unsucess_id, on = 'train_guest_id', how='leftanti').select(*cleaned_df.columns)
train_final = train_final.where('(change_assigned != transactiondate) or change_assigned is null')

In [0]:
train_tensor = []
label_tensor = []
length_tensor = []
id_tensor = []

features_scaled = []
for i in range(len(FEATURE_NAMES_RETURN)):
    features_scaled.append(FEATURE_NAMES_RETURN[i]+'_scaled')

for i in tqdm(range(365, 0, -1)):
    cus_list = train_final.filter(train_final.remaining_days == i).select('train_guest_id')
    temp_df = train_final.join(cus_list, (train_final.train_guest_id == cus_list.train_guest_id) & (train_final.remaining_days >= i), how = 'inner').drop(train_final.train_guest_id)
    temp_df = temp_df.withColumn('rn', F.row_number().over(Window.partitionBy('train_guest_id').orderBy('remaining_days'))).where('rn <= 6')
    if temp_df.count() > 0: train_np, label, length, train_id = create_sequences_and_labels(temp_df, features_scaled, 6)
    
    train_tensor.extend(train_np)
    label_tensor.extend(label)
    length_tensor.extend(length)
    id_tensor.extend(train_id)

In [0]:
len(train_tensor)